In [1]:
import asyncio
import logging
from datetime import datetime

import influxdb_client
import pandas as pd
from IPython.display import Markdown, display

from tastytrade.common.logging import setup_logging
from tastytrade.config import RedisConfigManager
from tastytrade.config.enumerations import Channels
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.messaging.processors import (
    RedisEventProcessor,
    TelegrafHTTPEventProcessor,
)
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

2026-01-10 03:50:47 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20260110.log


# Service Connections


In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

# Safely attach processors only if the router is initialized
router = dxlink.router
if router is None:
    raise RuntimeError("DXLink router not initialized after open()")
handlers_dict = getattr(router, "handler", None)
if handlers_dict is None:
    raise RuntimeError("DXLink router.handler mapping not initialized")
for h in handlers_dict.values():
    h.add_processor(TelegrafHTTPEventProcessor())
    h.add_processor(RedisEventProcessor())

2026-01-10 03:50:50 - INFO:tastytrade.config.manager:174:Initialized 19 variables from .env file in Redis
2026-01-10 03:50:50 - DEBUG:asyncio:848:Get address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32>
2026-01-10 03:50:50 - DEBUG:asyncio:858:Getting address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32> took 40.090ms: [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('170.76.246.67', 443))]
2026-01-10 03:50:50 - DEBUG:asyncio:535:<asyncio.sslproto.SSLProtocol object at 0x7be5c0064090> starts SSL handshake
2026-01-10 03:50:50 - DEBUG:asyncio:594:<asyncio.sslproto.SSLProtocol object at 0x7be5c0064090>: SSL handshake took 38.4 ms
2026-01-10 03:50:50 - DEBUG:asyncio:1121:<asyncio.TransportSocket fd=74, family=2, type=1, proto=6, laddr=('172.18.0.2', 54210), raddr=('170.76.246.67', 443)> connected to None:None: (<asyncio.sslproto._SSLProtocolTransport object at 0x7

2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:225:SETUP
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:7
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:9
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:11
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:99
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:236:FEED_CONFIG:5:COMPACT
2026-01-10 03:50:51 - INFO:tastytrade.messaging.handlers:236:FEED_CONFIG:7:COMPACT
2026-01-10 03:50:51 - INFO:tastytrade.messaging.han

# Market Data Subscriptions


In [3]:
start_time = datetime(2025, 12, 8)

symbols = ["BTC/USD:CXTALP", "NVDA", "AAPL", "QQQ", "SPY", "SPX"]  # , "/ESH25:XCME"]
# symbols = ["BTC/USD:CXTALP"]
# symbols = ["NVDA"]
# symbols = ["QQQ"]
# symbols = ["AAPL"]
# symbols = ["SPY"]
# symbols = ["SPX"]

intervals = ["1d", "1h", "30m", "15m", "5m", "m"]

# ticker subscriptions
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in symbols:
    for interval in intervals:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=60)

2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: NVDA
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: AAPL
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: QQQ
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: SPY
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: SPX
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=d}
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=h}
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=30m}
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=15m}
2026-01-10 03:51:01 - INFO:tastytrade.connections.sockets:236:A

In [4]:
# forward fill
symbols = ["BTC/USD:CXTALP", ".NVDA", "AAPL", "QQQ", "SPY", "SPX"] 

for symbol in symbols:
    for interval in intervals:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

2026-01-10 03:52:35 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2026-01-10 03:52:35 - INFO:root:124:Forward-fill added 92 events for BTC/USD:CXTALP{=m}
2026-01-10 03:52:35 - INFO:root:49:No CandleEvent data found for .NVDA{=d} in the last 5 days
2026-01-10 03:52:35 - WARNING:root:138:No data found for .NVDA{=1d} in the last 5 days
2026-01-10 03:52:35 - INFO:root:49:No CandleEvent data found for .NVDA{=h} in the last 5 days
2026-01-10 03:52:35 - WARNING:root:138:No data found for .NVDA{=1h} in the last 5 days
2026-01-10 03:52:35 - INFO:root:49:No CandleEvent data found for .NVDA{=30m} in the last 5 days
2026-01-10 03:52:35 - WARNING:root:138:No data found for .NVDA{=30m} in the last 5 days
2026-01-10 03:52:35 - INFO:root:49:No CandleEvent data found for .NVDA{=15m} in the last 5 days
2026-01-10 03:52:35 - WARNING:root:138:No data found for .NVDA{=15m} in the last 5 days
2026-01-10 03:52:35 - INFO:root:49:No CandleEvent data found for .NVDA{=5m} in the last 5 days


# Check | Market Data feeds


In [5]:
symbol = "SPX{=m}"
# symbol = "BTC/USD:CXTALP{=5m}"
# symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
router = dxlink.router
if router is None or getattr(router, "handler", None) is None:
    raise RuntimeError("Router or handler map not initialized")
handlers_dict = router.handler
candle_handler = handlers_dict.get(Channels.Candle)
if candle_handler is None:
    raise KeyError("Candle channel handler not available")
feed_proc = candle_handler.processors.get("feed")
if feed_proc is None:
    raise KeyError("'feed' processor missing on Candle handler")
frames = getattr(feed_proc, "frames", {})
if symbol not in frames:
    raise KeyError(f"No frames for symbol {symbol}")
(frames[f"{symbol}"].tail(5).sort(by="time", descending=True))

**Candle Feed:** SPX{=m}

eventSymbol,time,eventFlags,index,sequence,count,open,high,low,close,volume,bidVolume,askVolume,openInterest,vwap,impVolatility
str,datetime[μs],i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""SPX{=m}""",2026-01-09 21:51:00,4,7593482680176476160,0,1,6966.28,6966.28,6966.28,6966.28,null,null,null,null,null,0.14
"""SPX{=m}""",2026-01-09 21:20:00,0,7593474691537305600,0,1,6966.28,6966.28,6966.28,6966.28,null,null,null,null,null,0.14
"""SPX{=m}""",2026-01-09 21:19:00,0,7593474433839267840,0,60,6966.28,6966.28,6966.28,6966.28,null,null,null,null,null,0.14
"""SPX{=m}""",2026-01-09 21:18:00,0,7593474176141230080,0,60,6966.28,6966.28,6966.28,6966.28,null,null,null,null,null,0.14
"""SPX{=m}""",2026-01-09 21:17:00,0,7593473918443192320,0,60,6966.28,6966.28,6966.28,6966.28,null,null,null,null,null,0.14


In [6]:
display(Markdown(f"**Quote Feed:** {symbol}"))
router = dxlink.router
if router is None or getattr(router, "handler", None) is None:
    raise RuntimeError("Router or handler map not initialized")
quote_handler = router.handler.get(Channels.Quote)
if quote_handler is None:
    raise KeyError("Quote channel handler not available")
feed_proc = quote_handler.processors.get("feed")
if feed_proc is None:
    raise KeyError("'feed' processor missing on Quote handler")
feed_proc.pl

**Quote Feed:** SPX{=m}

eventSymbol,bidPrice,askPrice,bidSize,askSize
str,f64,f64,f64,f64
"""SPY""",693.74,695.08,null,null
"""AAPL""",259.35,263.31,null,null
"""SPX""",6938.44,6987.86,null,null
"""BTC/USD:CXTALP""",90027.86,90932.67,0.03,0.04
"""QQQ""",627.02,627.08,null,null
"""NVDA""",184.95,186.74,null,null


In [ ]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [ ]:
await dxlink.close()